In [ ]:
system_prompt = 'You are a programmer,you use python,sympy to solve problems,set variables,exprestions,set Interval to give range of variable,and use solve() function to calculation.only give a python code'

### load data

In [10]:
import json
from datasets import load_dataset, Dataset
with open('conic10k/test.json', 'r', encoding='utf-8') as file:
    raw_conic_tr = json.load(file)
datasett = load_dataset('json', data_files='conic10k/test.json', split=None)
# dataset1['train']['text']
# len(dataset1)
datasett['train'][0]['text']

Generating train split: 0 examples [00:00, ? examples/s]

'如果双曲线的两个焦点分别为$F_{1}(-3,0)$, $F_{2}(3,0)$ ,一条渐近线方程为$y=\\sqrt{2} x$，则该双曲线的方程为?'

### prompt to have answer

In [17]:
#init data

ans = {'Ai_answer':[],'real_answer':[],'primary_output':[]}
flag = 0
my_dataset = datasett
print(my_dataset)

DatasetDict({
    train: Dataset({
        features: ['query_spans', 'text', 'query_expressions', 'process', 'fact_spans', 'answer_expressions', 'fact_expressions'],
        num_rows: 2069
    })
})


In [18]:
#将大模型输出的结果中的代码提取并运行，将结果返回
import re
import sys
from io import StringIO
def to_py_and_run(text):
    pattern = r'```python\n(.*?)\n```'
    match = re.search(pattern, text, re.DOTALL)
    run_ans = 'None'
    if match:
        code_block = match.group(1).strip()  # 提取代码块并去除首尾空白
        
        # 截取最后一行提取代码结果
        last_line = code_block.rstrip().rsplit('\n', 1)[-1]

        # #最后一行可能是print(answer_expr),也可能是answer_expr，我们归一化处理
        ppattern = r"print\((.*)\)" 
        pmatch = re.search(ppattern, last_line)
        pmatch
        if pmatch:
            no_use=1 # jump
        else:
            code_block = code_block + '\n\nprint('+ last_line+')'
        
        # with open('outputcode.py', 'w') as file:
        #     file.write(code_block)  # 将代码写入文件
        # #为了保存输出，使用重定向


        old_stdout = sys.stdout
        new_stdout = StringIO()
        sys.stdout = new_stdout

        # 运行生成的Python文件 保存输出到返回值，并恢复输出定向
        try:
            exec(code_block,globals())
            output = new_stdout.getvalue()

        except:
            run_ans = 'code_error'
        else:
            # print('printsuccess')
            run_ans = output.strip('\n')
        sys.stdout = old_stdout
            
    return run_ans

In [31]:
ans['primary_output'] = ['empty' for _ in range(73)]
# ans['primary_output']

In [ ]:
# 第73个数据开始，保存了primaryoutput

In [39]:
from gradio_client import Client
client = Client("Qwen/Qwen2-72B-Instruct")
i=flag
while i < len(my_dataset['train']['text']) and i<500:
  mathproblem = my_dataset['train'][i]['text']
  problemans = my_dataset['train'][i]['answer_expressions']
  result = client.predict(
  query=mathproblem,
  history=[],
  system="You are a programmer,you use python,sympy to solve problems,set variables,set Interval to calculate the range of question,exprestions,and use solve() function to calculation.only give a python code.(code cannot have external inputs and graphical outputs)",
  api_name="/model_chat"
  )
  print('finish predict')
  ans['primary_output'].append(result[1][0][1])
  code_result = to_py_and_run(result[1][0][1])
  if code_result =='code_error':
    stop_point = 0
    stop_limit = 1
    while code_result =='code_error' and stop_point < stop_limit:
      stop_point = stop_point+1
      print('error time:',stop_point)
      new_result = client.predict(
        query='you gave me a code to solve a problem,code:'+code_result+'problem:'+mathproblem+'pleas debug the code,then only give me a corrected python code',
        history=[],
        system="You are a programmer,you use python,sympy to solve problems,set variables,set Interval to calculate the range of question,exprestions,and use solve() function to calculation.only give a python code.(code cannot have external inputs and graphical outputs)",
        api_name="/model_chat"
      )
      code_result = to_py_and_run(new_result[1][0][1])

  ans['Ai_answer'].append(code_result)
  ans['real_answer'].append(problemans)
  print(i)
  print(mathproblem)
  print(code_result)
  print(problemans)
  with open('test_predict.json', 'w', encoding='utf-8') as file:
    json.dump(ans, file)
  i = i+1
  flag = i

Loaded as API: https://qwen-qwen2-72b-instruct.hf.space ✔
finish predict
405
已知椭圆$\frac{x^{2}}{10-m}+\frac{y^{2}}{m-2}=1$的长轴在$y$轴上，若焦距为$4$，则$m$=?
14
8
finish predict
406
已知双曲线$C$: $\frac{x^{2}}{a^{2}}-\frac{y^{2}}{b^{2}}=1(a>0, b>0)$的焦点为$F_{1}$、$F_{2}$，且双曲线$C$上的点$P$满足$\overrightarrow{P F_{1}} \cdot \overrightarrow{P F_{2}}=0$ ,$|\overrightarrow{P F_{1}}|=3$ ,$|\overrightarrow{P F_{2}}|=4  $，则双曲线$C$的离心率为?
5
5
finish predict
407
抛物线$y^{2}=4 x$的焦点$F$的坐标为?点$F$到双曲线$x^{2}-y^{2}=1$的渐近线的距离为?
sqrt(2)/2
(1,0);sqrt(2)/2
finish predict
408
已知双曲线$C$: $\frac{x^{2}}{4}-\frac{y^{2}}{b^{2}}=1$的左右焦点为$F_{1}$、$F_{2}$，且$|F_{1} F_{2}|=6$，则$F_{1}$到一渐近线的距离为?
1.49534878122122*I
sqrt(5)
finish predict
409
抛物线$y^{2}=6 x$的焦点为$F$，过点$F$的直线与抛物线的两个交点为$A$、$B$，若$A B$中点横坐标为$2$，则$|A B|$=?
7.00000000000000
7
finish predict
410
双曲线$\frac{x^{2}}{4}-y^{2}=1$的焦点坐标是?
(sqrt(5), 0) (-sqrt(5), 0)
(pm*sqrt(5),0)
finish predict
error time: 1
411
已知双曲线的渐近线方程为$y=\pm x$，且过点$(1, \sqrt{2})$，则双曲线的标准方程为?
Eq(x**2 - y**2, 1)
y^2-x^2=1
finis

c:\Users\ye\AppData\Local\Programs\Python\Python311\Lib\site-packages\sympy\core\basic.py:1131: SymPyDeprecationWarning: 

Using non-Expr arguments in Add is deprecated (in this case, one of
the arguments has type 'Tuple').

If you really did intend to use a multiplication or addition operation with
this object, use the * or + operator instead.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#non-expr-args-deprecated
for details.

This has been deprecated since SymPy version 1.7. It
will be removed in a future version of SymPy.

  rv = self.func(*args)


483
已知抛物线$C$: $y^{2}=2 p x(p>0)$的焦点为$F$，过点$M(-1,4)$作$y$轴的垂线交抛物线$C$于点$A$，且满足$|A F|=|A M|$，设直线$A F$交抛物线$C$于另一点$B$，则点$B$的纵坐标为?
code_error
-1
finish predict
484
点$P$为椭圆$C$上一动点，过点$P$作以椭圆短轴为直径的圆的两条切线，切点分别为$M$、$N$，若$\angle M P N=60^{\circ}$，则椭圆$C$的离心率的取值范围是?
{0.816496580927726}
[sqrt(3)/2,1)
finish predict
485
已知双曲线$C$: $\frac{x^{2}}{16}-\frac{y^{2}}{4}=1$的左、右焦点分别为$F_{1}$、$F_{2}$，点$A$在双曲线$C$的左支上，且$|A F_{1}|=12$，则$|A F_{2}|$=?
4
20
finish predict
486
椭圆$\frac{x^{2}}{k+8}+\frac{y^{2}}{9}=1$的离心率$e=\frac{1}{2}$, 则$k$的值是?
4.00000000000000
{4,-5/4}
finish predict
error time: 1
487
已知抛物线$y^{2}=2 p x(p>0)$的焦点为$F(-1,0)$，直线$l$: $y=-x+m$与抛物线交于不同的两点$A$、$B$，若$0 \leq m<1$，则$\Delta F A B$的面积的最大值是?
code_error
8*sqrt(6)/9
finish predict
488
抛物线$x^{2}=4 y$的焦点坐标为?
(0, 1)
(0,1)
finish predict
489
以坐标原点为顶点，焦点在坐标轴上且经过点$(1,2)$的抛物线的方程是?
-4*x + y**2 = 0
x**2 - y/2 = 0
{(y^2=4*x), (x^2=y/2)}
finish predict
490
椭圆$\frac{x^{2}}{4}+y^{2}=1$的两个焦点$F_{1}$、$F_{2}$，过点$F_{1}$作垂直于$x$轴的直线与椭圆相交，其中一个交点为$P$，则$|P F_{2}|$=?
7/2
7/2
f

In [41]:
import pandas as pd
# 需要库：openpyxl
# 将字典转换为pandas DataFrame
memory_df = pd.DataFrame(ans)
 
# 将DataFrame写入Excel文件
# !!!!记得修改你的文件名
memory_df.to_excel('qwen2_codeversion_1.xlsx', index=1)

In [27]:
# query flag
flag

73